In [ ]:
!pip install "ray[default]"

In [34]:
!pip install "modin[ray]" # Install Modin dependencies and Ray to run on Ray

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 25.6 MB 11.7 MB/s            


In [1]:
import requests
import pprint
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import random
import networkx as nx
import matplotlib.pyplot as plt
from typing import List
from typing import Dict
from itertools import combinations
import time

In [2]:
import ray
ray.init(num_cpus=16)

2021-12-08 20:58:24,419	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.100',
 'raylet_ip_address': '192.168.1.100',
 'redis_address': '192.168.1.100:6379',
 'object_store_address': '/tmp/ray/session_2021-12-08_20-58-22_060472_81221/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-08_20-58-22_060472_81221/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-08_20-58-22_060472_81221',
 'metrics_export_port': 61171,
 'node_id': 'cde035ec86450de2f891503de8ccfd9c18d1de37633dadeac9f5bd89'}

In [35]:
import modin.pandas as pd

## Old

In [ ]:
def filter_handles(df: pd.DataFrame, handles=[]) -> pd.DataFrame:
  df = df[df['handle'].isin(handles)]
  return df

def sample_random_users(df: pd.DataFrame, N:int = 5) -> pd.DataFrame:
  handles = list(df['handle'].unique())
  return random.sample(handles, N)

In [ ]:
from functools import lru_cache

URL_USER_SUBMISSIONS = 'https://codeforces.com/api/user.status'

@lru_cache(maxsize = 128)
def get_all_problems(df):
    return df['problemID'].unique().tolist()

@lru_cache(maxsize = 128)
def get_all_user_problems(user_handle):
    res = requests.get(URL_USER_SUBMISSIONS,
                       params={'handle': user_handle, 'from': 1,
                       'to': 10000})
    df = pd.DataFrame.from_dict(res.json()['result'])
    df = df[(df['verdict'] == 'OK') | (df['verdict'] == 'WRONG_ANSWER')]
    df = df[['problem', 'verdict']]
    df['problem'] = df['problem'].apply(lambda x: str(x['contestId']) \
            + x['index'])
    df.drop_duplicates(subset=['problem'])
    return df

@lru_cache(maxsize = 128)
def get_solved_problems(df, user_handle):
  return df[df['handle'] == user_handle] 

@lru_cache(maxsize = 128)
def get_pending_problems(user_handle):
    all_problems = get_all_problems()
    solved_problems = get_solved_problems(user_handle)
    df = all_problems[~all_problems['problem'].isin(solved_problems['problem'])]
    return df

## Loading and Sampling Dataset (Diego Canez)

In [36]:
def get_all_user_problems(user_handle: str):
    URL_USER_SUBMISSIONS = 'https://codeforces.com/api/user.status'
    res = requests.get(URL_USER_SUBMISSIONS,
                       params={'handle': user_handle})
    df = pd.DataFrame.from_dict(res.json()['result'])
    df = df[df['verdict'] == 'OK']
    df = df[['problem']]
    df['problem'] = df['problem'].apply(lambda x: str(x['contestId']) + x['index'] if 'contestId' in x else None)
    df.dropna(inplace=True)
    df.drop_duplicates(subset=['problem'], inplace=True)
    return df['problem'].tolist()

def prepare_dataset(user_handles: List[str]):
  data = {}
  for handle in user_handles:
    data[handle] = get_all_user_problems(handle)
    time.sleep(1)
  return data

In [37]:
def get_users(k=None):
  URL = 'https://codeforces.com/api/user.ratedList?activeOnly=true'
  res = requests.get(URL)
  df = pd.DataFrame.from_dict(res.json()['result'])
  ans = df['handle'].tolist()
  if k is None:
    k = len(ans)
  return random.sample(ans, k)


## Creation of the Graph (Jorge Rebosio)

In [38]:
def add_edges(G: nx.Graph, problems_ix: List[int]):
  for u, v in combinations(problems_ix, 2):
    if (G.has_edge(u , v)):
      G.edges[u, v]['weight'] += 1 
    else:
      G.add_edge(u, v, weight=1)

def create_graph(user_problems_ix: Dict[str, List[int]]) -> nx.Graph:
  G = nx.Graph()
  for user, problems_ix in user_problems_ix.items():
    add_edges(G, problems_ix)
  return G

### Plotting Subgraph

In [39]:
# Sampling 10 nodes from graph G
def draw_subgraph(G: nx.Graph, N = 10, edge_attribute='weight'):
  H = G.subgraph(list(G.nodes)[:N])
  pos = nx.spring_layout(H)
  nx.draw(H, pos)
  edge_labels = nx.get_edge_attributes(H,edge_attribute)
  nx.draw_networkx_edge_labels(H, pos, edge_labels=edge_labels)
  plt.show()

## Building similarity matrix (Diego Canez)

### Similarity Metrics

#### Edge Weights (EW)
$$
EW(x, y) = A_{x, y}
$$

In [40]:
def grouped(l, group_size):
    for i in range(0, len(l), group_size):
        yield i, min(i+group_size, len(l))

@ray.remote
def ew_batch_compute_value(G: nx.Graph, l: int, r: int):
    return [(ew_compute_value(G, x, y), x, y) for x, y in list(G.edges)[l:r]]

def ew_compute_value(G: nx.Graph, x:int, y:int):
    return G[x][y]['weight']

def edge_weights(G: nx.Graph):
    N = len(G.nodes)
    EW = np.zeros((N, N))
    G_id = ray.put(G)
    ans = [ew_batch_compute_value.remote(G_id, l, r) for l, r in grouped(G.edges(), 5000)]
    ans = ray.get(ans)
    for batch in ans:
        for val, x, y in batch:
            EW[x, y] = EW[x, y] = val
    return EW

#### Weighted Common Neighbors (WCN)
$$
WCN(x, y) = \Sigma_{z \in N(x) \cap N(y)} A_{x, z} + A_{z, y}
$$

In [41]:
def grouped(l, group_size):
    for i in range(0, len(l), group_size):
        yield i, min(i+group_size, len(l))

@ray.remote
def wcn_batch_compute_value(G: nx.Graph, l: int, r: int):
    return [(wcn_compute_value(G, x, y), x, y) for x, y in list(G.edges)[l:r]]

def wcn_compute_value(G: nx.Graph, x:int, y:int):
    val = 0.0
    for z in nx.common_neighbors(G, x, y):
        val += G[x][z]['weight'] + G[z][y]['weight']
    return val

def weighted_common_neighbors(G: nx.Graph, attr='weight'):
    N = len(G)
    WCN = np.zeros((N, N))
    G_id = ray.put(G)
    ans = [wcn_batch_compute_value.remote(G_id, l, r) for l, r in grouped(G.edges(), 5000)]
    ans = ray.get(ans)
    for batch in ans:
        for val, x, y in batch:
            WCN[x, y] = WCN[y, x] = val
    return WCN

## Produce Recommendations (Maria Lovaton)

Source: https://sci-hub.se/https://doi.org/10.1007/978-3-319-61030-6_7

In [42]:
def recommend(user_handle, sim, user_problems_ix, problems_ix, problems, k=None):
  N = len(problems_ix)
  solved_problems = user_problems_ix[user_handle]
  pending_problems = list(set(range(N)) - set(problems_ix))
  P = [0.0] * N
  for pi in solved_problems:
    for pj in pending_problems:
      if P[pj] < sim[pi, pj]:
        P[pj] = sim[pi, pj]
  ans = list(enumerate(P))
  ans.sort(key=lambda index_score: index_score[1], reverse=True)
  ans = [(problems[ix], score) for ix, score in ans]
  if k is None:
    return ans
  else:
    k = min(len(ans), max(0, k))
    return ans[:k]

## Execution

In [7]:
users = ['dgcnz'] + get_users(k=5)
print(users) 

['dgcnz', 'Yinch', 'ginious', 'Gornak40', 'VLADOSIO', 'Elkhiat']


In [43]:
%%time
user_problems = prepare_dataset(users)
problems = list({problem for problem_list in user_problems.values() for problem in problem_list})
problems_ix = {problemId : index for index, problemId in enumerate(problems)}
user_problems_ix = {user : [problems_ix[problem] for problem in problems] for user, problems in user_problems.items()}

To request implementation, send an email to feature_requests@modin.org.


CPU times: user 1.76 s, sys: 259 ms, total: 2.02 s
Wall time: 13.8 s


In [9]:
len(problems)

1373

In [10]:
%%time
G = create_graph(user_problems_ix)

CPU times: user 587 ms, sys: 40.3 ms, total: 627 ms
Wall time: 623 ms


In [32]:
%%time
sim = weighted_common_neighbors(G)
# sim = edge_weights(G)

CPU times: user 4.41 s, sys: 663 ms, total: 5.07 s
Wall time: 2min 59s


In [31]:
nx.density(G)

0.6287321712578486

In [30]:
recommend('dgcnz', sim, user_problems_ix, problems_ix, problems, k=5)

[('1610A', 2995.0),
 ('791A', 2995.0),
 ('1474B', 2993.0),
 ('1433D', 2993.0),
 ('1312B', 2993.0)]